# 1. Read JSON file

In [43]:
import json

#Read JSON data into the datastore variable
filename = 'online course data.json'
with open(filename, 'r', encoding='UTF-8') as f:
    onlineCourses_JSONarr = json.load(f)

#Use the new datastore datastructure
print(len(onlineCourses_JSONarr))
print(onlineCourses_JSONarr[0].keys())

826
dict_keys(['cTitle', 'cProvider', 'cPhotoLink', 'cDirectLink', 'cDescription', 'cWeekSection', 'cAuthor', 'cLanguage', 'tmpCateg', 'tmpId'])


In [154]:
for i in range(0,10):
    print(onlineCourses_JSONarr[i]['cTitle'])

Machine Learning
用Python玩转数据 Data Processing Using Python
Building a Data Science Team
Command Line Tools for Genomic Data Science
A Crash Course in Data Science
Data Science in Real Life
Materials Data Sciences and Informatics
Computer Science:  Algorithms, Theory, and Machines
Data Science Methodology
Data Science Ethics


In [112]:
onlineCourses_JSONarr[0]['cWeekSection']

"[{'name': 'Introduction', 'description': 'Welcome to Machine Learning! In this module, we introduce the core idea of teaching a computer to learn concepts using data—without being explicitly programmed. The Course Wiki is under construction. Please visit the resources tab for the most complete and up-to-date information.', 'timeCommitment': 7088953, 'id': 'aOf6j', 'lessonIds': ['GOIXz', 'URH5n', '4Jpz0'], 'slug': 'introduction'}, {'name': 'Linear Regression with One Variable', 'description': 'Linear regression predicts a real-valued output based on an input value. We discuss the application of linear regression to housing price prediction, present the notion of a cost function, and introduce the gradient descent method for learning.', 'timeCommitment': 7500147, 'id': 'tN10A', 'lessonIds': ['emCIn', '7AyTq', 'bmUPd'], 'slug': 'linear-regression-with-one-variable'}, {'name': 'Linear Algebra Review', 'description': 'This optional module provides a refresher on linear algebra concepts. Ba

# 2. Helper functions

In [255]:
def addProfessor(prof,courseName):
    '''Check if professor exists in db, creates new professor if not and the TEACHES relationship with course'''
    query = 'MATCH(professor:Professor{name:\''+prof+'\'}) return count(professor)> 0 as exists'
    data = graph.run(query)

    for d in data:
        result = d.items()[0]
        if (result==('exists', True)):
            #print('Prof Node ' + prof+'  already exists')
            pass
        else:
            try:
                query = 'CREATE (professor:Professor {name:\'' + prof + '\'})' 
                graph.run(query)
            except:
                print('ERROR CREATING: Professor node with name ' + prof )
    
        query = 'MERGE (professor:Professor {name: \"' + prof + '\"})' +\
                'MERGE (course:Course {title: \"' + courseName + '\"})' +\
                'CREATE UNIQUE (professor)-[:TEACHES]->(course)'
        graph.run(query)
                

In [256]:
def addLanguage(lang ,courseName):
    '''Check if professor exists in db, creates new language if not and the LANGUAGE relationship with course'''
    query = 'MATCH(language:Language{name:\'' + lang + '\'}) return count(language)> 0 as exists'
    data = graph.run(query)

    for d in data:
        result = d.items()[0]
        if (result==('exists', True)):
            #print('Lang Node ' + lang+'  already exists')
            pass
        else:
            try:
                query = 'CREATE (language:Language {name:\'' + lang + '\'})' 
                graph.run(query)
            except:
                print('ERROR CREATING: Language node with name ' + lang )
    
        query = 'MERGE (language:Language {name: \"' + lang + '\"})' +\
                'MERGE (course:Course {title: \"' + courseName + '\"})' +\
                'CREATE UNIQUE (course)-[:LANGUAGE]->(language)'
        graph.run(query)
                

In [263]:
def addCategory(cat ,courseName):
    '''Check if professor exists in db, creates new category if not and the category relationship with course'''
    query = 'MATCH(category:Category{number:\'' + cat + '\'}) return count(category)> 0 as exists'
    data = graph.run(query)

    for d in data:
        result = d.items()[0]
        if (result==('exists', True)):
            #print('cat Node ' + cat +'  already exists')
            pass
        else:
            try:
                query = 'CREATE (category:Category {number:\'' + cat + '\'})' 
                graph.run(query)
            except:
                print('ERROR CREATING: category node with number ' + cat )
    
        query = 'MERGE (category:Category {number: \"' + cat + '\"})' +\
                'MERGE (course:Course {title: \"' + courseName + '\"})' +\
                'CREATE UNIQUE (course)-[:IS_CATEGORY]->(category)'
        graph.run(query)
                

# 3. Connect to Neo4j DB

In [274]:
#connect and clean db
from py2neo import Graph

graph = Graph("http://localhost:7474", user='EvaArevalo', password='yhwh123')
graph.delete_all()

In [271]:
#Create uniqueness constraints
graph.run('CREATE CONSTRAINT ON (professor:Professor) ASSERT professor.name IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (course:Course) ASSERT course.title IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (language:Language) ASSERT language.language IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (category:Category) ASSERT category.number IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (provider:Provider) ASSERT provider.name IS UNIQUE')

In [275]:
#Create Nodes
import re
#dict_keys
#dict_keys(['cTitle', 'cProvider', 'cPhotoLink', 'cDirectLink', 'cDescription', 'cWeekSection', 
#'cAuthor', 'cLanguage', 'tmpCateg', 'tmpId'])

for course in onlineCourses_JSONarr[0:3]:
    #Create Course node, with properties: cTitle', 'cPhotoLink', 'cDirectLink', 'cDescription', 'cWeekSection' 
    #'tmpId' and relationships: 'cProvider',  'cAuthor', 'cLanguage', 'tmpCateg'
    description = re.sub('\"','\'',course['cDescription'])
    weekSection = re.sub('\"','\'',str(course['cWeekSection']))
    query = 'CREATE (course:Course {title:\'' + course['cTitle'] + '\', ' +\
                                    'photoLink:\"' + course['cPhotoLink'] + '\", ' +\
                                    'directLink:\"' + course['cDirectLink'] + '\", ' +\
                                    'weekSection:\"' + weekSection + '\", ' +\
                                    'description:\"' + description + '\"})' 
    #'weekSection:\"' + str(course['cWeekSection']) + '\", ' +\
    graph.run(query)
    
    #Add relationships and other nodes
    addLanguage(course['cLanguage'], course['cTitle'])
    for cat in course['tmpCateg']:
        addCategory(str(cat),course['cTitle'])
    for prof in course['cAuthor']:
        addProfessor(prof, course['cTitle'])


# 3. Draw graph

In [276]:
import scripts.vis
from scripts.vis import draw

options = {"Professor": "name", "Course": "title", "Language":"language", "Category":"number"}
draw(graph, options)